In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

# データセットの準備
# Fashion-MNISTデータセットをダウンロードして前処理
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

# データローダーの設定
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

# クラスラベルの定義
classes = [
    "Tシャツ/トップ", "ズボン", "プルオーバー", "ドレス", "コート",
    "サンダル", "シャツ", "スニーカー", "バッグ", "アンクルブーツ"
]

# デバイスの設定（GPU/MPS/CPU）
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

# ニューラルネットワークモデルの定義
class NeuralNetwork(nn.Module):
    def __init__(self, dropout_rate=0.5):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),  # ドロップアウトを追加
            
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),  # ドロップアウトを追加
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

# 学習用の関数定義
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"損失: {loss:>7f}  [{current:>5d}/{size:>5d}]")

# テスト用の関数定義
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"テスト結果: \n 精度: {(100*correct):>0.1f}%, 平均損失: {test_loss:>8f} \n")


In [ ]:
# モデルのインスタンス化
model = NeuralNetwork(dropout_rate=0.5).to(device)

# 損失関数とオプティマイザの設定
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

# 学習の実行
epochs = 5
for t in range(epochs):
    print(f"エポック {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

In [ ]:
test_iter = iter(test_dataloader)
testX, testy = next(test_iter)
pred = model(testX.to(device))
plt.imshow(testX[0].numpy().reshape(28, 28))
print('True      Label:', classes[testy[0].item()])
print('Estimated Label:', classes[pred.argmax(1)[0]])

In [ ]:
# モデルの保存
torch.save(model.state_dict(), "model.pth")

# モデルの読み込み
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))